<a href="https://colab.research.google.com/github/zolfaShefreie/MCI-Chatbot/blob/main/ASR_BOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#installations

In [2]:
!apt-get update && apt-get install -y libsndfile1 ffmpeg
!pip install Cython
!pip install nemo_toolkit[all]
!pip install wget
!apt-get install sox libsndfile1 ffmpeg
!pip install unidecode
!pip install matplotlib>=3.3.2

## Install NeMo
BRANCH = 'r1.11.0'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

## Grab the config we'll use in this example
!mkdir configs
!wget -P configs/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/asr/conf/config.yaml

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 https://developer.downlo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsndfile1 is already the newest version (1.0.28-4ubuntu0.18.04.2).
ffmpeg is already the newest version (7:3.4.11-0ubuntu0.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3
Suggested packages:
  file libsox-fmt-all
The following NEW packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3 sox
0 upgraded, 8 newly installed, 0 to remove and 20 not upgraded.
Need to get 760 kB of archives.
After this operation, 6,717 kB of additional disk space will be used.
Get:1 http://archive

In [3]:
!pip install pyTelegramBotAPI

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 210 kB 8.6 MB/s 
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.7.0-py3-none-any.whl size=192826 sha256=efa6e27a23706ae734ffbfcdddbc60a230d749590592c17c2b3d691cda983402
  Stored in directory: /root/.cache/pip/wheels/ba/13/da/8abf941f7cf9f993cde6118a56a7c24e12ed791507acd8ea39
Successfully built pyTelegramBotAPI


#imports

In [4]:
import telebot
import os
import subprocess
import librosa
import nemo.collections.asr as nemo_asr

[NeMo W 2022-09-28 10:17:10 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.


#TLEGRAM BOT

In [5]:
API_KEY = "5786378772:AAH710mta2_NqZ5HTm76dY5oa63BQUUBsMg"

In [13]:
asr_model = nemo_asr.models.ASRModel.from_pretrained("stt_en_jasper10x5dr", map_location='cpu')
last_model_path = "/content/drive/MyDrive/mci/models/last-epoch=7.ckpt"
if os.path.exists(last_model_path):
    asr_model = asr_model.load_from_checkpoint(last_model_path)

[NeMo I 2022-09-28 10:23:17 cloud:56] Found existing object /root/.cache/torch/NeMo/NeMo_1.11.0/stt_en_jasper10x5dr/856ae08d5c4bd78b5e27f696e96f7aab/stt_en_jasper10x5dr.nemo.
[NeMo I 2022-09-28 10:23:17 cloud:62] Re-using file from: /root/.cache/torch/NeMo/NeMo_1.11.0/stt_en_jasper10x5dr/856ae08d5c4bd78b5e27f696e96f7aab/stt_en_jasper10x5dr.nemo
[NeMo I 2022-09-28 10:23:17 common:910] Instantiating model from pre-trained checkpoint


[NeMo W 2022-09-28 10:23:39 modelPT:143] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data2/voices/train_1k.json
    sample_rate: 16000
    labels:
    - ' '
    - a
    - b
    - c
    - d
    - e
    - f
    - g
    - h
    - i
    - j
    - k
    - l
    - m
    - 'n'
    - o
    - p
    - q
    - r
    - s
    - t
    - u
    - v
    - w
    - x
    - 'y'
    - z
    - ''''
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    
[NeMo W 2022-09-28 10:23:39 modelPT:150] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: /data2/vo

[NeMo I 2022-09-28 10:23:39 features:223] PADDING: 16
[NeMo I 2022-09-28 10:23:50 save_restore_connector:243] Model EncDecCTCModel was successfully restored from /root/.cache/torch/NeMo/NeMo_1.11.0/stt_en_jasper10x5dr/856ae08d5c4bd78b5e27f696e96f7aab/stt_en_jasper10x5dr.nemo.


[NeMo W 2022-09-28 10:24:08 modelPT:143] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /content/drive/MyDrive/mci/manifests/fa/commonvoice_train_manifest_processed.json,/content/drive/MyDrive/mci/manifests/fa/commonvoice_dev_manifest_processed.json
    sample_rate: 16000
    labels:
    - ِ
    - p
    - m
    - c
    - ئ
    - ژ
    - ﺱ
    - ش
    - گ
    - i
    - ۀ
    - ج
    - ذ
    - ā
    - خ
    - ء
    - پ
    - ﻮ
    - ً
    - b
    - م
    - ص
    - 'y'
    - ر
    - ض
    - z
    - ی
    - ُ
    - ﻢ
    - ؤ
    - ة
    - ح
    - ى
    - ا
    - س
    - ت
    - ﻡ
    - t
    - ب
    - ف
    - غ
    - و
    - f
    - ٌ
    - 'n'
    - ' '
    - َ
    - ل
    - ﺩ
    - چ
    - a
    - ّ
    - ظ
    - ق
    - ث
    - d
    - ْ
    - ﺍ
    - د
    - ک
    - e
    - ٔ
    - r
    - ع
    - ز
    - ه
    - ط
    - š


[NeMo I 2022-09-28 10:24:08 features:223] PADDING: 16


In [14]:
asr_model.cfg.labels

['ِ', 'p', 'm', 'c', 'ئ', 'ژ', 'ﺱ', 'ش', 'گ', 'i', 'ۀ', 'ج', 'ذ', 'ā', 'خ', 'ء', 'پ', 'ﻮ', 'ً', 'b', 'م', 'ص', 'y', 'ر', 'ض', 'z', 'ی', 'ُ', 'ﻢ', 'ؤ', 'ة', 'ح', 'ى', 'ا', 'س', 'ت', 'ﻡ', 't', 'ب', 'ف', 'غ', 'و', 'f', 'ٌ', 'n', ' ', 'َ', 'ل', 'ﺩ', 'چ', 'a', 'ّ', 'ظ', 'ق', 'ث', 'd', 'ْ', 'ﺍ', 'د', 'ک', 'e', 'ٔ', 'r', 'ع', 'ز', 'ه', 'ط', 'š', 'ن', 's']

In [15]:


class ASREchoBot:
    """
    Echo Bot management
    """
    BOT = telebot.TeleBot(API_KEY)
    HELP_TEXT = "You can control bot by sending these commands:\n\n\n" \
                "/start_echo : start echo process\n" \
                "/stop_echo: start echo process\n" \
                "/asr: use asr service for convert voice to text"

    MODEL = asr_model


    @staticmethod
    @BOT.message_handler(commands=['start', 'help'])
    def help(message):
        """
        function for handle /start and /help command
        :param message:
        :return:
        """
        ASREchoBot.BOT.send_message(message.chat.id, ASREchoBot.HELP_TEXT)

    @staticmethod
    @BOT.message_handler(commands=['start_echo'])
    def start_echo(message):
        """
        function for handle /start_echo command
        :param message:
        :return:
        """
        sent_msg = ASREchoBot.BOT.send_message(message.chat.id, "start Echo\nEnter your message")
        ASREchoBot.BOT.register_next_step_handler(sent_msg, ASREchoBot.echo_message_handler)

    @staticmethod
    @BOT.message_handler(commands=['stop_echo'])
    def stop_echo(message):
        """
        function for handle /stop_echo command
        :param message:
        :return:
        """
        ASREchoBot.BOT.send_message(message.chat.id, "Echo process is stopped")
    
    @staticmethod
    @BOT.message_handler(commands=['asr'])
    def asr(message):
        """
        function for handle /asr command 
        :param message:
        :return:
        """
        sent_msg = ASREchoBot.BOT.send_message(message.chat.id, "Send a voice")
        ASREchoBot.BOT.register_next_step_handler(sent_msg, ASREchoBot.echo_voice_handler)
    
    @staticmethod
    def convert_to_text(wav_file_path: str):
        """
        using nemo model to convert wave file to text
        :param wav_file_path: path of wav file
        :return: result of asr process
        """
        return ASREchoBot.MODEL.transcribe(paths2audio_files=[wav_file_path],
                                 batch_size=1)[0]

    @staticmethod
    def convert_oga_to_wav(file_path: str):
        """
        convert oga format to wav format
        :param file_path: path of oga file
        :return: path of new file format
        """
        dst = f"{file_path.split('.')[0]}.wav"
        process = subprocess.run(['ffmpeg', '-i', file_path, dst])
        return dst
    
    @staticmethod
    def echo_voice_handler(message):
        """
        function for manage the messages after /asr
        to convert voice to text
        :param message:
        :return:
        """
        if message.voice is None:
            ASREchoBot.BOT.reply_to(message, "Invalid Message Format: please send a voice")
        else:
          file_info = ASREchoBot.BOT.get_file(message.voice.file_id)

          #save file to system
          downloaded_file = ASREchoBot.BOT.download_file(file_info.file_path)
          src = file_info.file_path.split("/")[-1]
          with open(src, 'wb') as new_file:
              new_file.write(downloaded_file)
          
          file_path = ASREchoBot.convert_oga_to_wav(src)

          text = ASREchoBot.convert_to_text(file_path)
          ASREchoBot.BOT.reply_to(message, text)         

    @staticmethod
    def echo_message_handler(message):
        """
        function for manage the messages after /start_echo
        if the message is /stop_echo the stop_echo handler is called else send the same message to user
        :param message:
        :return:
        """
        if message.text != "/stop_echo":
            sent_msg = ASREchoBot.BOT.send_message(message.chat.id, message.text)
            ASREchoBot.BOT.register_next_step_handler(sent_msg, ASREchoBot.echo_message_handler)
        else:
            ASREchoBot.stop_echo(message)

    @classmethod
    def run(cls):
        cls.BOT.polling()


In [16]:
if __name__ == "__main__":
    ASREchoBot.run()

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

[NeMo W 2022-09-28 10:24:50 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
      warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
    


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

[NeMo W 2022-09-28 10:25:44 nemo_logging:349] /usr/local/lib/python3.7/dist-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
      warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
    
